In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import snscrape.modules.twitter as sntwitter
import numpy as np
import pandas as pd
from scipy.special import softmax
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import seaborn as sns
import yfinance as yf
import csv

Things to update:
- Filenames for 1.1 snscrape
- Saving scraper file in right place
- Maintaining and using blacklist
-rename master files and update for old_... vars

In [21]:
def update():
    def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)+timedelta(hours=t.minute//30))

    def timedeltas(start='2022-06-24 06:00:00',last="2022-07-01 06:00:00",hour_delta=8,min_gap=10):

    #     end = pd.Timestamp(start)+timedelta(minutes=min_gap)
    #     end = end.isoformat("_", "seconds")



        if last == None:
            current = datetime.datetime.now()
            current.isoformat()
            current = pd.Timestamp(current)
            current = current.tz_localize(None)
        else:
            current = pd.Timestamp(last)

        ts = pd.Timestamp(start)
        ts = hour_rounder(ts+timedelta(hours=8))
        ts = pd.Timestamp(ts)
        ts = ts.tz_localize(None)

        start_list = []
        end_list = []
        start_list.append(ts.isoformat("_", "seconds")+"_UTC")
        end = ts + timedelta(minutes=min_gap)
        end_list.append(end.isoformat("_", "seconds")+"_UTC")

        while (ts+timedelta(hours=hour_delta))<(current-timedelta(minutes=min_gap)):
            ts = ts + timedelta(hours=hour_delta)
            es = ts + timedelta(minutes=min_gap)
            start_list.append(ts.isoformat("_", "seconds")+"_UTC")
            end_list.append(es.isoformat("_", "seconds")+"_UTC")

        return start_list, end_list


    def append_to_csv(tweet, fileName):

        #A counter variable
        counter = 0

        #Open OR create the target CSV file
        csvFile = open(fileName, "a", newline="", encoding='utf-8')
        csvWriter = csv.writer(csvFile)

        datetime_date = tweet.date

        # 2. username
        username = tweet.user.username

        # 8. Tweet text
        text = tweet.content

        # Assemble all data in a list
        res = [datetime_date, username, text]

        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

        # When done, close the CSV file
        csvFile.close()


        return

    def snscrape(num_tweets=149,start_date='2023-03-05T06:00:00.000Z',end_date='2023-03-06T06:00:00.000Z', hour_delta=8,min_gap=10,filename='run1.csv',make_csv=True):

        start_list, end_list = timedeltas(start=start_date,last=end_date,hour_delta=hour_delta,min_gap=min_gap)

        # Create file
        csvFile = open(filename, "a", newline="", encoding='utf-8')
        csvWriter = csv.writer(csvFile)

        #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
        csvWriter.writerow(['datetime', 'username', 'text'])
        csvFile.close()

        # Using TwitterSearchScraper to scrape data and append tweets to list
        for i in range(len(start_list)):
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'#bitcoin since:{start_list[i]} until:{end_list[i]} lang:en').get_items()):
                if i>num_tweets:
                    break
                append_to_csv(tweet, filename)

        # Creating a dataframe from the tweets list above
    #     tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

        if make_csv:
            return pd.read_csv(filename)
        else:
            return

    old_tweet_data = pd.read_csv("~/Code/giadapi/crypto/training_data/complete_tweet.csv",lineterminator='\n',index_col=0)
    old_vol_data = pd.read_csv("~/Code/giadapi/crypto/training_data/complete_vol.csv",lineterminator='\n',index_col=0)

    datetime_idx = old_tweet_data.columns.get_loc("datetime")
    last_time = old_tweet_data.iloc[-1,datetime_idx]
    scraped_tweets = snscrape(num_tweets=149,start_date=last_time,end_date=None, hour_delta=8,min_gap=10,filename=f'scraped_{last_time[:10]}.csv',make_csv=True)

    scraped_tweets = scraped_tweets.rename(columns={'text\r':'text'})

    scraped_tweets['datetime'] = pd.to_datetime(scraped_tweets['datetime'])
    scraped_tweets['date'] = scraped_tweets['datetime'].dt.date

    def remove_spam(data):
        dup_tweets = data[data['text'].duplicated()]
        blacklist = []
        for name in dup_tweets['username']:
            blacklist.append(name)
        blacklist = list(set(blacklist))
        def to_remove(x):
            if x in blacklist:
                return True
            else:
                return False
        data['to_remove'] = data['username'].apply(to_remove)
        mask = data['to_remove']
        data2 = data[~mask]
        return data2

    def preprocess(text):
        new_text = []
        text = str(text)
        text = text.replace("\n", " ")
        for t in text.split(" "):
            t = '' if t.startswith('@') and len(t) > 1 else t
            t = '' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    scraped_tweets = remove_spam(scraped_tweets)
    data =  scraped_tweets[['text', 'date','datetime','username']]

    #Create a dummy data
    data['process_text'] = data.text
    data['negative_bert'] = data.text
    data['neutral_bert'] = data.text
    data['positive_bert'] = data.text

    #use the preprocess_2 to clean the data
    data['process_text'] = data['text'].apply(preprocess)

    #Load Model
    #Model 2b - Use the bert model, with full scores
    MODEL_bert = f"ElKulako/cryptobert"
    tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
    tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
    config_bert = AutoConfig.from_pretrained(MODEL_bert)


    # PT
    model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
    model_bert.config.max_position_embeddings = 512

    def scores_bert(sample_text):
        encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
        output_bert = model_bert(**encoded_input_bert)
        scores_bert = output_bert[0][0].detach().numpy()
        scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
        return scores_bert

    data['text'] = data['process_text'].apply(scores_bert)
    data.reset_index(inplace=True)
    
    for i in range(len(data)):    
        data['negative_bert'][i] = data['text'][i][0]
        data['neutral_bert'][i] = data['text'][i][1]
        data['positive_bert'][i] = data['text'][i][2]

    new_df = data.copy()
    new_df.drop("text",axis=1,inplace=True)

    new_tweet_data = new_df.rename(columns={'process_text':'text'})
    new_tweet_data[["datetime", "username", "text", "date", "negative_bert", "neutral_bert", "positive_bert"]].to_csv(f"new_tweet_data_{last_time[:10]}.csv")

    def tweet_daily_rate(tweet_data_original):
        tweet_data = tweet_data_original.copy()

        tweet_data['datetime'] = pd.to_datetime(tweet_data['datetime'])
        tweet_data['hour'] = tweet_data['datetime'].dt.hour
        tweet_data['date'] = tweet_data['datetime'].dt.date

        daily_tweets = tweet_data.groupby(['date', 'hour']).agg({'negative_bert':'mean',
                                                     'neutral_bert':'mean',
                                                     'positive_bert':'mean',
                                                     'text':'count',
                                                     'datetime': lambda x: x.max() - x.min()})



        daily_tweets['datetime_seconds'] = daily_tweets['datetime'].dt.total_seconds()
        daily_tweets['tweets_rate'] = daily_tweets['text']/(daily_tweets['datetime_seconds'])
        daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
        daily_tweets_col['tweets_rate']*86400

        cut_off =daily_tweets.columns.get_loc("positive_bert")
        vol_data = daily_tweets.iloc[:,:cut_off+1]

        vol_data.reset_index(inplace=True)
        vol_data.drop('hour',axis=1,inplace=True)

        vol_data = vol_data.groupby(['date']).agg({'negative_bert':'mean',
                                            'neutral_bert':'mean',
                                            'positive_bert':'mean',
                                            })

        daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
        vol_data['daily_tweets'] = daily_tweets_col['tweets_rate']*86400
        vol_data.reset_index(inplace=True)
        return vol_data

    # Pull btc price
    def load_historic_data(symbol, start_date_str, end_date_str, period, interval):
        try:
            df = yf.download(symbol, start=start_date_str, end=end_date_str, period=period, interval=interval)
            #  Add symbol

            df["Symbol"] = symbol
            df['price_change'] = df['Close'].pct_change()
    #         print(df)
            return df

        except:
            print(f'Error loading stock data for + {symbol}')
            return None

        # Convert tweet_data to vol_data (Adding moving averages,btc)
        # Automate saving of the file?
    def daily_data(tweet_data, filename, start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d'):

        vol_data = tweet_daily_rate(tweet_data)

        if start_date_str == None:
            v_date = vol_data.columns.get_loc("date")
            start_date_str = vol_data.iloc[0,v_date]

        vol_data['MA7_Sentiment'] = vol_data['positive_bert'].rolling(window=8).mean()
        vol_data['MA20_Sentiment'] = vol_data['positive_bert'].rolling(window=21).mean()
        vol_data['MA50_Sentiment'] = vol_data['positive_bert'].rolling(window=51).mean()

        if end_date_str == None:
            today = datetime.date.today()
            today_str = datetime.datetime.strftime(today, "%Y-%m-%d")
            BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, today_str, period, interval))
        else:
            BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, end_date_str, period, interval))                    
    #     print(BTC_price)
        vol_data['BTC_price']=BTC_price.reset_index()['Adj Close']
        vol_data['BTC_trading_volume']=BTC_price.reset_index()['Volume']

        vol_data = vol_data[['date','negative_bert', 'neutral_bert', 'positive_bert',
           'daily_tweets', 'MA7_Sentiment', 'MA20_Sentiment',
           'MA50_Sentiment', 'BTC_price','BTC_trading_volume']]


        vol_data.to_csv(f'{filename}.csv')
        return vol_data

    fully_combined = pd.concat([old_tweet_data,new_tweet_data])
    fully_combined.reset_index(drop=True,inplace=True)

    new_vol_master = daily_data(fully_combined, f'new_vol_master_{last_time[:10]}.csv', start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d')

    fully_combined.to_csv('training_data/complete_tweet.csv')
    new_vol_master.to_csv("training_data/complete_vol.csv")

    print("Update Completed")

    return

    
    
    

In [22]:
update()

/var/folders/m3/wrwzlkcn30j_t2hg5r1616g80000gn/T/ipykernel_97936/2746789774.py:56: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  text = tweet.content
/var/folders/m3/wrwzlkcn30j_t2hg5r1616g80000gn/T/ipykernel_97936/2746789774.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['negative_bert'][i] = data['text'][i][0]
/var/folders/m3/wrwzlkcn30j_t2hg5r1616g80000gn/T/ipykernel_97936/2746789774.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['neutral_bert'][i] = data['text'][i][1]
/var/folders/m3/wrwzlkcn30j_t2hg5r1616g80000gn/T/ipykernel_97936/2746789774.py:173: SettingWithCo

[*********************100%***********************]  1 of 1 completed
Update Completed


In [20]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/suraj/code/giadapi/crypto


In [24]:
pd.read_csv("~/Code/giadapi/crypto/training_data/complete_tweet.csv",lineterminator='\n',index_col=0)

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648
1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093
2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644
3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960
4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495
...,...,...,...,...,...,...,...
350295,2023-03-15 14:04:04+00:00,ThisIsNateMori,"Run on the banks 2023 🏦 Be your own bank, #Bi...",2023-03-15,0.000687,0.332631,0.666683
350296,2023-03-15 14:04:01+00:00,BlakeMillardCFA,"What about investing in #bitcoin? Joe, bitco...",2023-03-15,0.000144,0.699408,0.300448
350297,2023-03-15 14:04:00+00:00,CryptoLifer33,3 things I regretted not doing once I did them...,2023-03-15,0.009185,0.442825,0.547990
350298,2023-03-15 14:03:57+00:00,BitcoinBible_,Do you pay tax on Bitcoin? #bitcoin #BTC #tax...,2023-03-15,0.008515,0.528597,0.462889


In [23]:
old_tweet_data = pd.read_csv("~/Code/giadapi/crypto/training_data/test_complete_tweet.csv",lineterminator='\n',index_col=0)
old_vol_data = pd.read_csv("~/Code/giadapi/crypto/training_data/test_complete_vol.csv",lineterminator='\n',index_col=0)
old_tweet_data

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert,index
0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648,NaN
1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093,NaN
2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644,NaN
3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960,NaN
4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495,NaN
...,...,...,...,...,...,...,...,...
350545,2023-03-16 06:00:02+00:00,CarbonaraIndex,"The price of Spaghetti alla Carbonara is 44,98...",2023-03-16,0.020797,0.737039,0.242164,256.0
350546,2023-03-16 06:00:02+00:00,topcryptostats,"Bitcoin - BTC Price: $24,449.60 Change in 1h: ...",2023-03-16,0.002927,0.824798,0.172275,257.0
350547,2023-03-16 06:00:01+00:00,CryptoHawkPro,Which side are you on? #Bitcoin or #Bank? #c...,2023-03-16,0.008992,0.702474,0.288534,258.0
350548,2023-03-16 06:00:00+00:00,crypto_maven,The US govt did not create #Bitcoin . If you ...,2023-03-16,0.037996,0.627166,0.334838,259.0


# Run Snscrape

## Import functions

In [ ]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

def timedeltas(start='2022-06-24 06:00:00',last="2022-07-01 06:00:00",hour_delta=8,min_gap=10):

#     end = pd.Timestamp(start)+timedelta(minutes=min_gap)
#     end = end.isoformat("_", "seconds")
    
    

    if last == None:
        current = datetime.now()
        current.isoformat()
        current = pd.Timestamp(current)
        current = current.tz_localize(None)
    else:
        current = pd.Timestamp(last)

    ts = pd.Timestamp(start)
    ts = hour_rounder(ts+timedelta(hours=8))
    ts = pd.Timestamp(ts)
    ts = ts.tz_localize(None)
    
    start_list = []
    end_list = []
    start_list.append(ts.isoformat("_", "seconds")+"_UTC")
    end = ts + timedelta(minutes=min_gap)
    end_list.append(end.isoformat("_", "seconds")+"_UTC")

    while (ts+timedelta(hours=hour_delta))<(current-timedelta(minutes=min_gap)):
        ts = ts + timedelta(hours=hour_delta)
        es = ts + timedelta(minutes=min_gap)
        start_list.append(ts.isoformat("_", "seconds")+"_UTC")
        end_list.append(es.isoformat("_", "seconds")+"_UTC")

    return start_list, end_list


def append_to_csv(tweet, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    datetime_date = tweet.date

    # 2. username
    username = tweet.user.username

    # 8. Tweet text
    text = tweet.content

    # Assemble all data in a list
    res = [datetime_date, username, text]

    # Append the result to the CSV file
    csvWriter.writerow(res)
    counter += 1

    # When done, close the CSV file
    csvFile.close()


    return

def snscrape(num_tweets=149,start_date='2023-03-05T06:00:00.000Z',end_date='2023-03-06T06:00:00.000Z', hour_delta=8,min_gap=10,filename='run1.csv',make_csv=True):

    start_list, end_list = timedeltas(start=start_date,last=end_date,hour_delta=hour_delta,min_gap=min_gap)

    # Create file
    csvFile = open(filename, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['datetime', 'username', 'text'])
    csvFile.close()

    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i in range(len(start_list)):
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'#bitcoin since:{start_list[i]} until:{end_list[i]} lang:en').get_items()):
            if i>num_tweets:
                break
            append_to_csv(tweet, filename)

    # Creating a dataframe from the tweets list above
#     tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

    if make_csv:
        return pd.read_csv(filename)
    else:
        return

## Import Current Data - update filenames

In [4]:
old_tweet_data = pd.read_csv("~/Code/giadapi/crypto/streamlit_data/master_tweet_data.csv",lineterminator='\n',index_col=0)
old_tweet_data

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648
1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093
2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644
3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960
4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495
...,...,...,...,...,...,...,...
2513,2023-03-14 22:00:34+00:00,antoniomotgg,"""I love the smell of Napalm in the morning!"" -...",2023-03-14,0.018969,0.422259,0.558773
2514,2023-03-14 22:00:33+00:00,ashleygrilo12,They are about to turn the money printers on a...,2023-03-14,0.000570,0.312646,0.686783
2515,2023-03-14 22:00:30+00:00,BtcPulse,BTC hourly update $24658.30 | -0.54%📉 $B...,2023-03-14,0.368925,0.523527,0.107548
2516,2023-03-14 22:00:29+00:00,Bcubeai,#Bitcoin rallied 10% in one day. The next ones...,2023-03-14,0.000383,0.454934,0.544683


In [5]:
old_vol_data = pd.read_csv("~/Code/giadapi/crypto/streamlit_data/master_vol_data.csv",lineterminator='\n',index_col=0)
old_vol_data


,date,negative_bert,neutral_bert,positive_bert,daily_tweets,MA7_Sentiment,MA20_Sentiment,MA50_Sentiment,BTC_price,BTC_trading_volume
0,2021-01-01,0.085275,0.489181,0.425545,12561.859509,NaN,NaN,NaN,29374.152344,40730301359
1,2021-01-02,0.143365,0.439075,0.417560,30124.352145,NaN,NaN,NaN,32127.267578,67865420765
2,2021-01-03,0.148220,0.432023,0.419757,26167.331282,NaN,NaN,NaN,32782.023438,78665235202
3,2021-01-04,0.101558,0.507526,0.390916,18041.199084,NaN,NaN,NaN,31971.914062,81163475344
4,2021-01-05,0.088530,0.517456,0.394014,17473.551568,NaN,NaN,NaN,33992.429688,67547324782
...,...,...,...,...,...,...,...,...,...,...
798,2023-03-10,0.122003,0.496804,0.381193,23029.804160,0.365374,0.357392,0.361947,20187.244141,39578257695
799,2023-03-11,0.118417,0.483141,0.398441,21168.432091,0.371627,0.359184,0.362839,20632.410156,30180288176
800,2023-03-12,0.137051,0.479557,0.383392,18621.077853,0.372813,0.359771,0.362830,22163.949219,29279035521
801,2023-03-13,0.126823,0.496415,0.376762,24769.727538,0.375752,0.360913,0.362850,24197.533203,49466362688


## Run Snscrape to get till last date

In [6]:
datetime_idx = old_tweet_data.columns.get_loc("datetime")
last_time = old_tweet_data.iloc[-1,datetime_idx]
last_time[:10]
last_time

'2023-03-14 22:00:29+00:00'

In [ ]:
scraped_tweets = snscrape(num_tweets=149,start_date=last_time,end_date=None, hour_delta=8,min_gap=10,filename=f'scraped_{last_time[:10]}.csv',make_csv=True)
scraped_tweets




# Processing

## Processing

In [ ]:
scraped_tweets = pd.read_csv(f'scraped_{last_time[:10]}.csv',lineterminator='\n')
scraped_tweets = scraped_tweets.rename(columns={'text\r':'text'})
scraped_tweets

In [ ]:
scraped_tweets['datetime'] = pd.to_datetime(scraped_tweets['datetime'])
scraped_tweets['date'] = scraped_tweets['datetime'].dt.date
scraped_tweets

In [ ]:
def remove_spam(data):
    dup_tweets = data[data['text'].duplicated()]
    blacklist = []
    for name in dup_tweets['username']:
        blacklist.append(name)
    blacklist = list(set(blacklist))
    def to_remove(x):
        if x in blacklist:
            return True
        else:
            return False
    data['to_remove'] = data['username'].apply(to_remove)
    mask = data['to_remove']
    data2 = data[~mask]
    return data2

In [ ]:
def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
scraped_tweets = remove_spam(scraped_tweets)
data =  scraped_tweets[['text', 'date','datetime','username']]
data

In [ ]:
#Create a dummy data
data['process_text'] = data.text
data['negative_bert'] = data.text
data['neutral_bert'] = data.text
data['positive_bert'] = data.text

#use the preprocess_2 to clean the data
data['process_text'] = data['text'].apply(preprocess)

## Loading Model

In [ ]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [ ]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

## Running Model

In [ ]:
data['text'] = data['process_text'].apply(scores_bert)

In [ ]:
for i in range(len(data)):    
    data['negative_bert'][i] = data['text'][i][0]
    data['neutral_bert'][i] = data['text'][i][1]
    data['positive_bert'][i] = data['text'][i][2]

# Formatting

In [ ]:
new_df = data.copy()
new_df

In [ ]:
new_df.drop("text",axis=1,inplace=True)

In [ ]:
new_df = new_df.rename(columns={'process_text':'text'})
new_df[["datetime", "username", "text", "date", "negative_bert", "neutral_bert", "positive_bert"]].to_csv(f"new_tweet_data_{last_time[:10]}.csv")


# Aggregation

In [7]:
new_tweet_data = pd.read_csv(f"new_tweet_data_{last_time[:10]}.csv",lineterminator='\n',index_col=0)
new_tweet_data

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,2023-03-15 06:09:53+00:00,pumpcoininfo,#DFUSDT ✅ Increased % 3.03 in 1 min 💵 Price - ...,2023-03-15,0.070416,0.632298,0.297286
1,2023-03-15 06:09:45+00:00,BitcoinJesus666,I need #bitcoin to go lower I'll add $50 then ...,2023-03-15,0.020003,0.489932,0.490065
2,2023-03-15 06:09:38+00:00,JackWorld_,Dragon #bitcoin #Tradingview,2023-03-15,0.005613,0.646720,0.347667
3,2023-03-15 06:09:30+00:00,nergizy32132110,Now I'm ready to launch the #Dex #Terraport 🚀 ...,2023-03-15,0.000091,0.167781,0.832128
4,2023-03-15 06:09:27+00:00,elitecryptotool,#bitcoin daily candles ✨,2023-03-15,0.013300,0.612623,0.374077
...,...,...,...,...,...,...,...
266,2023-03-15 14:04:04+00:00,ThisIsNateMori,"Run on the banks 2023 🏦 Be your own bank, #Bi...",2023-03-15,0.000687,0.332631,0.666683
267,2023-03-15 14:04:01+00:00,BlakeMillardCFA,"What about investing in #bitcoin? Joe, bitco...",2023-03-15,0.000144,0.699408,0.300448
268,2023-03-15 14:04:00+00:00,CryptoLifer33,3 things I regretted not doing once I did them...,2023-03-15,0.009185,0.442825,0.547990
269,2023-03-15 14:03:57+00:00,BitcoinBible_,Do you pay tax on Bitcoin? #bitcoin #BTC #tax...,2023-03-15,0.008515,0.528597,0.462889


In [30]:
# Concern of order of data too


# Get daily tweets from tweet_data
def tweet_daily_rate(tweet_data_original):
    tweet_data = tweet_data_original.copy()
    
    tweet_data['datetime'] = pd.to_datetime(tweet_data['datetime'])
    tweet_data['hour'] = tweet_data['datetime'].dt.hour
    tweet_data['date'] = tweet_data['datetime'].dt.date
    
    daily_tweets = tweet_data.groupby(['date', 'hour']).agg({'negative_bert':'mean',
                                                 'neutral_bert':'mean',
                                                 'positive_bert':'mean',
                                                 'text':'count',
                                                 'datetime': lambda x: x.max() - x.min()})
    
    
    
    daily_tweets['datetime_seconds'] = daily_tweets['datetime'].dt.total_seconds()
    daily_tweets['tweets_rate'] = daily_tweets['text']/(daily_tweets['datetime_seconds'])
    daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
    daily_tweets_col['tweets_rate']*86400
    
    cut_off =daily_tweets.columns.get_loc("positive_bert")
    vol_data = daily_tweets.iloc[:,:cut_off+1]

    vol_data.reset_index(inplace=True)
    vol_data.drop('hour',axis=1,inplace=True)

    vol_data = vol_data.groupby(['date']).agg({'negative_bert':'mean',
                                        'neutral_bert':'mean',
                                        'positive_bert':'mean',
                                        })
    
    daily_tweets_col = daily_tweets.groupby('date').agg({'tweets_rate':'mean'})
    vol_data['daily_tweets'] = daily_tweets_col['tweets_rate']*86400
    vol_data.reset_index(inplace=True)
    return vol_data

# Pull btc price
def load_historic_data(symbol, start_date_str, end_date_str, period, interval):
    try:
        df = yf.download(symbol, start=start_date_str, end=end_date_str, period=period, interval=interval)
        #  Add symbol
    
        df["Symbol"] = symbol
        df['price_change'] = df['Close'].pct_change()
#         print(df)
        return df
    
    except:
        print(f'Error loading stock data for + {symbol}')
        return None

# Convert tweet_data to vol_data (Adding moving averages,btc)
# Automate saving of the file?
def daily_data(tweet_data, filename, start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d'):
    
    vol_data = tweet_daily_rate(tweet_data)
    
    if start_date_str == None:
        v_date = vol_data.columns.get_loc("date")
        start_date_str = vol_data.iloc[0,v_date]
    
    vol_data['MA7_Sentiment'] = vol_data['positive_bert'].rolling(window=8).mean()
    vol_data['MA20_Sentiment'] = vol_data['positive_bert'].rolling(window=21).mean()
    vol_data['MA50_Sentiment'] = vol_data['positive_bert'].rolling(window=51).mean()
    
    if end_date_str == None:
        today = datetime.date.today()
        today_str = datetime.datetime.strftime(today, "%Y-%m-%d")
        BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, today_str, period, interval))
    else:
        BTC_price = pd.DataFrame(load_historic_data(symbol, start_date_str, end_date_str, period, interval))                    
#     print(BTC_price)
    vol_data['BTC_price']=BTC_price.reset_index()['Adj Close']
    vol_data['BTC_trading_volume']=BTC_price.reset_index()['Volume']
    
    vol_data = vol_data[['date','negative_bert', 'neutral_bert', 'positive_bert',
       'daily_tweets', 'MA7_Sentiment', 'MA20_Sentiment',
       'MA50_Sentiment', 'BTC_price','BTC_trading_volume']]
    
    
    vol_data.to_csv(f'{filename}.csv')
    return vol_data

In [14]:
fully_combined = pd.concat([old_tweet_data,new_tweet_data])
fully_combined.reset_index(drop=True,inplace=True)
fully_combined

,datetime,username,text,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01 22:09:59+00:00,sivilianlife,What's the chances @Grayscale is hedging short...,2021-01-01,0.002040,0.951312,0.046648
1,2021-01-01 22:09:39+00:00,LudicrousSpeed9,@CNBC Buy #Bitcoin to hedge against the result...,2021-01-01,0.004157,0.400750,0.595093
2,2021-01-01 22:09:35+00:00,SergioBowers1,@APompliano @pepsi @MonsterEnergy $DEO $KO e...,2021-01-01,0.000248,0.582107,0.417644
3,2021-01-01 22:09:21+00:00,bitvestor,Closed another chunk of this long now. Remaind...,2021-01-01,0.000974,0.807066,0.191960
4,2021-01-01 22:09:18+00:00,FROMKAIWITHLOVE,When you dig deeper into #kardiachain company ...,2021-01-01,0.000056,0.223449,0.776495
...,...,...,...,...,...,...,...
350295,2023-03-15 14:04:04+00:00,ThisIsNateMori,"Run on the banks 2023 🏦 Be your own bank, #Bi...",2023-03-15,0.000687,0.332631,0.666683
350296,2023-03-15 14:04:01+00:00,BlakeMillardCFA,"What about investing in #bitcoin? Joe, bitco...",2023-03-15,0.000144,0.699408,0.300448
350297,2023-03-15 14:04:00+00:00,CryptoLifer33,3 things I regretted not doing once I did them...,2023-03-15,0.009185,0.442825,0.547990
350298,2023-03-15 14:03:57+00:00,BitcoinBible_,Do you pay tax on Bitcoin? #bitcoin #BTC #tax...,2023-03-15,0.008515,0.528597,0.462889


In [31]:
new_vol_master = daily_data(fully_combined, f'new_vol_master_{last_time[:10]}.csv', start_date_str=None,symbol='BTC-USD', end_date_str=None, period='1d', interval='1d')
new_vol_master

[*********************100%***********************]  1 of 1 completed


,date,negative_bert,neutral_bert,positive_bert,daily_tweets,MA7_Sentiment,MA20_Sentiment,MA50_Sentiment,BTC_price,BTC_trading_volume
0,2021-01-01,0.085275,0.489181,0.425545,12561.859509,NaN,NaN,NaN,29374.152344,4.073030e+10
1,2021-01-02,0.143365,0.439075,0.417560,30124.352145,NaN,NaN,NaN,32127.267578,6.786542e+10
2,2021-01-03,0.148220,0.432023,0.419757,26167.331282,NaN,NaN,NaN,32782.023438,7.866524e+10
3,2021-01-04,0.101558,0.507526,0.390916,18041.199084,NaN,NaN,NaN,31971.914062,8.116348e+10
4,2021-01-05,0.088530,0.517456,0.394014,17473.551568,NaN,NaN,NaN,33992.429688,6.754732e+10
...,...,...,...,...,...,...,...,...,...,...
799,2023-03-11,0.118417,0.483141,0.398441,21168.432091,0.371627,0.359184,0.362839,20632.410156,3.018029e+10
800,2023-03-12,0.137051,0.479557,0.383392,18621.077853,0.372813,0.359771,0.362830,22163.949219,2.927904e+10
801,2023-03-13,0.126823,0.496415,0.376762,24769.727538,0.375752,0.360913,0.362850,24197.533203,4.946636e+10
802,2023-03-14,0.101771,0.503114,0.395115,45600.116185,0.378136,0.363325,0.363191,24746.074219,5.462223e+10


In [32]:
fully_combined.to_csv(f'complete_tweet_{last_time[:10]}.csv')
new_vol_master.to_csv(f"complete_vol_{last_time[:10]}.csv")